# Exposure Universe Analysis: Data Retrieval, Real Returns & Visualization

This notebook demonstrates the complete workflow for:
1. Loading the exposure universe configuration
2. Fetching total returns for all exposures
3. Retrieving inflation data from FRED
4. Converting nominal returns to real returns
5. Visualizing the results with various charts

**Author**: Portfolio Optimizer Team  
**Date**: July 2025

## 1. Setup and Imports

In [1]:
# Standard imports
import sys
import os
import warnings
from datetime import datetime, timedelta

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Project imports
sys.path.append('..')
from src.data import (
    ExposureUniverse, 
    TotalReturnFetcher, 
    FREDDataFetcher,
    ReturnEstimationFramework
)

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.float_format', '{:.4f}'.format)

print("✅ Setup complete!")

✅ Setup complete!


## 2. Load Exposure Universe Configuration

In [2]:
# Load the exposure universe
config_path = '../config/exposure_universe.yaml'
universe = ExposureUniverse.from_yaml(config_path)

print(f"Loaded exposure universe: {universe}")
print(f"\nCategories ({len(universe.get_all_categories())}):")
for category in universe.get_all_categories():
    exposures = universe.get_exposures_by_category(category)
    print(f"  • {category}: {len(exposures)} exposures")
    for exp in exposures:
        print(f"    - {exp.name} ({exp.id})")

Loaded exposure universe: ExposureUniverse(16 exposures, 5 categories)

Categories (5):
  • real_assets: 4 exposures
    - Real Estate (real_estate)
    - Broad Commodities (commodities)
    - Gold (gold)
    - Treasury Inflation-Protected Securities (tips)
  • equity_beta: 5 exposures
    - US Large Cap Equity Beta (us_large_equity)
    - US Small Cap Equity Beta (us_small_equity)
    - Developed Ex-US Large Cap Equity Beta (intl_developed_large_equity)
    - Developed Ex-US Small Cap Equity Beta (intl_developed_small_equity)
    - Emerging Markets Equity Beta (emerging_equity)
  • nominal_fixed_income: 4 exposures
    - Cash/Risk-Free Rate (cash_rate)
    - Short-Term US Treasuries (short_ust)
    - Broad US Treasuries (broad_ust)
    - Dynamic Global Bonds (dynamic_global_bonds)
  • factor_style: 2 exposures
    - Factor/Style - Equities (factor_style_equity)
    - Factor/Style - Other (factor_style_other)
  • alternatives: 1 exposures
    - Trend Following (trend_following)


## 3. Set Analysis Parameters

In [3]:
# Define analysis period
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)  # 5 years of history

# Analysis parameters
frequency = "monthly"  # daily, weekly, or monthly
inflation_series = "cpi_all"  # cpi_all, cpi_core, pce, pce_core

print(f"Analysis Period: {start_date.date()} to {end_date.date()}")
print(f"Frequency: {frequency}")
print(f"Inflation Series: {inflation_series}")

Analysis Period: 2020-07-06 to 2025-07-05
Frequency: monthly
Inflation Series: cpi_all


## 4. Fetch Total Returns for All Exposures

In [4]:
# Initialize data fetchers
total_return_fetcher = TotalReturnFetcher()
fred_fetcher = FREDDataFetcher()

# Fetch returns for all exposures
print("Fetching total returns for all exposures...")
universe_returns = total_return_fetcher.fetch_universe_returns(
    universe, start_date, end_date, frequency
)

# Summary of results
successful = sum(1 for r in universe_returns.values() if r['success'])
print(f"\n✅ Successfully fetched: {successful}/{len(universe_returns)} exposures")

# Show which exposures succeeded/failed
print("\nData Availability:")
for exp_id, result in universe_returns.items():
    exposure = universe.get_exposure(exp_id)
    status = "✅" if result['success'] else "❌"
    impl = result['implementation'] if result['success'] else "Failed"
    print(f"{status} {exposure.name:40} | {impl}")

Fetching total returns for all exposures...

✅ Successfully fetched: 16/16 exposures

Data Availability:
✅ US Large Cap Equity Beta                 | ETF average of ['SPY', 'IVV', 'VOO']
✅ US Small Cap Equity Beta                 | ETF average of ['IWM', 'IJR', 'VB']
✅ Developed Ex-US Large Cap Equity Beta    | ETF average of ['EFA', 'IEFA', 'VEA']
✅ Developed Ex-US Small Cap Equity Beta    | ETF average of ['SCZ', 'IEUS', 'VSS']
✅ Emerging Markets Equity Beta             | ETF average of ['EEM', 'IEMG', 'VWO']
✅ Factor/Style - Equities                  | Fund QMNIX
✅ Factor/Style - Other                     | Fund QSPIX
✅ Trend Following                          | Fund average of ['ABYIX', 'AHLIX', 'AQMNX', 'ASFYX']
✅ Cash/Risk-Free Rate                      | Rate series DGS3MO from FRED
✅ Short-Term US Treasuries                 | ETF average of ['SHY', 'SCHO', 'VGSH']
✅ Broad US Treasuries                      | ETF average of ['IEF', 'IEI', 'GOVT']
✅ Dynamic Global Bonds          

## 5. Fetch Inflation Data and Calculate Real Returns

In [5]:
# Fetch inflation data
print(f"Fetching {inflation_series} inflation data...")
inflation_index = fred_fetcher.fetch_inflation_data(
    start_date, end_date, inflation_series, frequency
)

if not inflation_index.empty:
    # Calculate inflation rates - FIXED: Use annualize=False for monthly data
    # This prevents the severe negative real returns issue
    inflation_rates = fred_fetcher.calculate_inflation_rate(
        inflation_index, periods=1, annualize=False  # CRITICAL FIX: False for monthly returns
    )
    
    print(f"✅ Fetched {len(inflation_rates)} inflation observations")
    print(f"   Latest monthly inflation: {inflation_rates.iloc[-1]:.4%}")
    print(f"   Average monthly inflation: {inflation_rates.mean():.4%}")
    print(f"   Approximate annual inflation: {inflation_rates.mean() * 12:.2%}")
    print(f"   Min/Max: {inflation_rates.min():.4%} / {inflation_rates.max():.4%}")
else:
    print("❌ No inflation data available")
    inflation_rates = pd.Series(dtype=float)

Fetching cpi_all inflation data...
✅ Fetched 57 inflation observations
   Latest monthly inflation: 0.0809%
   Average monthly inflation: 0.3732%
   Approximate annual inflation: 4.48%
   Min/Max: -0.0500% / 1.2952%


In [6]:
# Convert nominal returns to real returns
real_returns_data = {}
nominal_returns_data = {}

for exp_id, result in universe_returns.items():
    if result['success'] and not result['returns'].empty:
        nominal_returns = result['returns']
        nominal_returns_data[exp_id] = nominal_returns
        
        if not inflation_rates.empty:
            # Convert to real returns
            real_returns = fred_fetcher.convert_to_real_returns(
                nominal_returns, inflation_rates, method="exact"
            )
            if not real_returns.empty:
                real_returns_data[exp_id] = real_returns

print(f"✅ Converted {len(real_returns_data)} exposures to real returns")

# Create DataFrames
nominal_returns_df = pd.DataFrame(nominal_returns_data)
real_returns_df = pd.DataFrame(real_returns_data)

✅ Converted 16 exposures to real returns


## 6. Calculate Summary Statistics

In [9]:
# Calculate annualized statistics
def calculate_annualized_stats(returns_df, frequency='monthly'):
    if frequency == 'daily':
        factor = 252
    elif frequency == 'weekly':
        factor = 52
    elif frequency == 'monthly':
        factor = 12
    else:
        factor = 1
    
    stats = pd.DataFrame()
    stats['Ann. Return'] = returns_df.mean() * factor
    stats['Ann. Volatility'] = returns_df.std() * np.sqrt(factor)
    stats['Sharpe Ratio'] = stats['Ann. Return'] / stats['Ann. Volatility']
    stats['Min Return'] = returns_df.min()
    stats['Max Return'] = returns_df.max()
    stats['Skewness'] = returns_df.skew()
    stats['Kurtosis'] = returns_df.kurtosis()
    
    # Add category information
    stats['Category'] = stats.index.map(
        lambda x: universe.get_exposure(x).category if universe.get_exposure(x) else 'Unknown'
    )
    
    return stats

# Calculate for both nominal and real returns
nominal_stats = calculate_annualized_stats(nominal_returns_df, frequency)
real_stats = calculate_annualized_stats(real_returns_df, frequency)

# Display comparison
print("📊 Real vs Nominal Return Statistics\n")
comparison = pd.DataFrame({
    'Nominal Return': nominal_stats['Ann. Return'],
    'Real Return': real_stats['Ann. Return'],
    'Inflation Impact': nominal_stats['Ann. Return'] - real_stats['Ann. Return'],
    'Real Volatility': real_stats['Ann. Volatility'],
    'Real Sharpe': real_stats['Sharpe Ratio']
})

# Sort by real return
comparison = comparison.sort_values('Real Return', ascending=False)
print(comparison.round(3))

📊 Real vs Nominal Return Statistics

                             Nominal Return  Real Return  Inflation Impact  \
factor_style_other                   0.1900       0.1490            0.0410   
factor_style_equity                  0.1760       0.1440            0.0320   
us_large_equity                      0.1580       0.0940            0.0650   
commodities                          0.1380       0.0760            0.0620   
gold                                 0.1120       0.0720            0.0400   
us_small_equity                      0.1260       0.0620            0.0640   
intl_developed_large_equity          0.1160       0.0600            0.0560   
intl_developed_small_equity          0.1040       0.0390            0.0640   
real_estate                          0.0810       0.0370            0.0440   
emerging_equity                      0.0670       0.0040            0.0630   
trend_following                      0.0440      -0.0020            0.0450   
cash_rate                  

## 7. Visualization: Performance Overview

In [10]:
# Create a comprehensive performance chart
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Real Returns by Category', 'Risk-Return Scatter', 
                    'Return Distribution', 'Rolling Sharpe Ratios'),
    specs=[[{'type': 'bar'}, {'type': 'scatter'}],
           [{'type': 'box'}, {'type': 'scatter'}]]
)

# 1. Bar chart of real returns by category
for category in real_stats['Category'].unique():
    mask = real_stats['Category'] == category
    fig.add_trace(
        go.Bar(
            x=real_stats[mask].index,
            y=real_stats[mask]['Ann. Return'] * 100,
            name=category,
            text=real_stats[mask]['Ann. Return'].apply(lambda x: f'{x:.1%}'),
            textposition='auto',
        ),
        row=1, col=1
    )

# 2. Risk-Return Scatter
for category in real_stats['Category'].unique():
    mask = real_stats['Category'] == category
    fig.add_trace(
        go.Scatter(
            x=real_stats[mask]['Ann. Volatility'] * 100,
            y=real_stats[mask]['Ann. Return'] * 100,
            mode='markers+text',
            name=category,
            text=real_stats[mask].index,
            textposition='top center',
            marker=dict(size=10)
        ),
        row=1, col=2
    )

# 3. Box plot of return distributions
for exp_id in real_returns_df.columns[:10]:  # Limit to first 10 for readability
    fig.add_trace(
        go.Box(
            y=real_returns_df[exp_id] * 100,
            name=exp_id,
            showlegend=False
        ),
        row=2, col=1
    )

# 4. Rolling Sharpe ratios (select exposures)
selected_exposures = ['us_large_equity', 'broad_ust', 'real_estate', 'gold']
for exp_id in selected_exposures:
    if exp_id in real_returns_df.columns:
        rolling_mean = real_returns_df[exp_id].rolling(12).mean() * 12
        rolling_std = real_returns_df[exp_id].rolling(12).std() * np.sqrt(12)
        rolling_sharpe = rolling_mean / rolling_std
        
        fig.add_trace(
            go.Scatter(
                x=rolling_sharpe.index,
                y=rolling_sharpe,
                name=exp_id,
                mode='lines'
            ),
            row=2, col=2
        )

# Update layout
fig.update_layout(
    height=800,
    showlegend=True,
    title_text="Exposure Universe Performance Analysis (Real Returns)",
    title_font_size=20
)

fig.update_xaxes(title_text="Exposure", row=1, col=1)
fig.update_yaxes(title_text="Annual Return (%)", row=1, col=1)
fig.update_xaxes(title_text="Annual Volatility (%)", row=1, col=2)
fig.update_yaxes(title_text="Annual Return (%)", row=1, col=2)
fig.update_xaxes(title_text="Exposure", row=2, col=1)
fig.update_yaxes(title_text="Monthly Return (%)", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_yaxes(title_text="Rolling Sharpe Ratio", row=2, col=2)

fig.show()

## 8. Cumulative Performance Comparison

In [11]:
# Calculate cumulative returns
def calculate_cumulative_returns(returns_df):
    return (1 + returns_df).cumprod()

# Select key exposures for comparison
key_exposures = [
    'us_large_equity', 'us_small_equity', 'intl_developed_large_equity',
    'emerging_equity', 'broad_ust', 'real_estate', 'gold', 'commodities'
]

# Filter available exposures
available_key_exposures = [exp for exp in key_exposures if exp in real_returns_df.columns]

# Calculate cumulative returns for both nominal and real
nominal_cumulative = calculate_cumulative_returns(nominal_returns_df[available_key_exposures])
real_cumulative = calculate_cumulative_returns(real_returns_df[available_key_exposures])

# Create the plot
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Nominal Cumulative Returns', 'Real Cumulative Returns'),
    vertical_spacing=0.1
)

# Add nominal returns
for col in nominal_cumulative.columns:
    fig.add_trace(
        go.Scatter(
            x=nominal_cumulative.index,
            y=nominal_cumulative[col],
            name=col,
            mode='lines',
            showlegend=True
        ),
        row=1, col=1
    )

# Add real returns
for col in real_cumulative.columns:
    fig.add_trace(
        go.Scatter(
            x=real_cumulative.index,
            y=real_cumulative[col],
            name=col,
            mode='lines',
            showlegend=False
        ),
        row=2, col=1
    )

# Add inflation cumulative impact
inflation_cumulative = calculate_cumulative_returns(inflation_rates)
fig.add_trace(
    go.Scatter(
        x=inflation_cumulative.index,
        y=inflation_cumulative,
        name='Cumulative Inflation',
        mode='lines',
        line=dict(dash='dash', color='red'),
        showlegend=True
    ),
    row=2, col=1
)

fig.update_layout(
    height=800,
    title_text="Cumulative Performance: Nominal vs Real Returns",
    title_font_size=20,
    hovermode='x unified'
)

fig.update_yaxes(title_text="Cumulative Return", type="log", row=1, col=1)
fig.update_yaxes(title_text="Cumulative Return", type="log", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

fig.show()

## 9. Correlation Analysis

In [12]:
# Calculate correlation matrix for real returns
correlation_matrix = real_returns_df.corr()

# Create a heatmap
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    colorscale='RdBu',
    zmid=0,
    text=correlation_matrix.round(2).values,
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title="Correlation")
))

fig.update_layout(
    title="Real Return Correlation Matrix",
    xaxis_tickangle=-45,
    width=1000,
    height=800
)

fig.show()

# Show highest and lowest correlations
# Get upper triangle of correlation matrix
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)
upper_triangle = correlation_matrix.where(mask)

# Flatten and sort
correlations = upper_triangle.stack()
correlations = correlations[correlations != 1.0]  # Remove diagonal

print("\n🔝 Highest Correlations:")
print(correlations.nlargest(5).round(3))

print("\n🔻 Lowest Correlations:")
print(correlations.nsmallest(5).round(3))


🔝 Highest Correlations:
intl_developed_large_equity  intl_developed_small_equity   0.9760
us_large_equity              real_estate                   0.8630
short_ust                    broad_ust                     0.8470
factor_style_equity          factor_style_other            0.8450
us_large_equity              us_small_equity               0.8430
dtype: float64

🔻 Lowest Correlations:
trend_following              broad_ust              -0.7170
                             short_ust              -0.6340
                             tips                   -0.6120
                             dynamic_global_bonds   -0.5390
intl_developed_large_equity  trend_following        -0.3520
dtype: float64


## 10. Inflation Impact Analysis

In [13]:
# Calculate inflation impact on returns
inflation_impact = pd.DataFrame({
    'Nominal Return': nominal_stats['Ann. Return'] * 100,
    'Real Return': real_stats['Ann. Return'] * 100,
    'Inflation Impact': (nominal_stats['Ann. Return'] - real_stats['Ann. Return']) * 100,
    'Category': real_stats['Category']
})

# Sort by inflation impact
inflation_impact = inflation_impact.sort_values('Inflation Impact', ascending=False)

# Create visualization
fig = go.Figure()

# Add bars for nominal returns
fig.add_trace(go.Bar(
    name='Nominal Return',
    x=inflation_impact.index,
    y=inflation_impact['Nominal Return'],
    marker_color='lightblue'
))

# Add bars for real returns
fig.add_trace(go.Bar(
    name='Real Return',
    x=inflation_impact.index,
    y=inflation_impact['Real Return'],
    marker_color='darkblue'
))

# Add line for inflation impact
fig.add_trace(go.Scatter(
    name='Inflation Impact',
    x=inflation_impact.index,
    y=inflation_impact['Inflation Impact'],
    mode='lines+markers',
    marker_color='red',
    yaxis='y2'
))

fig.update_layout(
    title='Inflation Impact on Asset Returns',
    xaxis_tickangle=-45,
    yaxis=dict(title='Annual Return (%)'),
    yaxis2=dict(title='Inflation Impact (%)', overlaying='y', side='right'),
    barmode='group',
    height=600,
    hovermode='x unified'
)

fig.show()

# Summary statistics
print("\n📊 Inflation Impact Summary:")
print(f"Average inflation impact: {inflation_impact['Inflation Impact'].mean():.2f}%")
print(f"\nMost affected by inflation:")
print(inflation_impact.head(3)[['Inflation Impact', 'Category']].round(2))
print(f"\nLeast affected by inflation:")
print(inflation_impact.tail(3)[['Inflation Impact', 'Category']].round(2))


📊 Inflation Impact Summary:
Average inflation impact: 5.03%

Most affected by inflation:
                             Inflation Impact     Category
us_large_equity                        6.4700  equity_beta
intl_developed_small_equity            6.4400  equity_beta
us_small_equity                        6.3700  equity_beta

Least affected by inflation:
                     Inflation Impact      Category
factor_style_other             4.0800  factor_style
gold                           4.0500   real_assets
factor_style_equity            3.1500  factor_style


## 11. Rolling Performance Analysis

In [14]:
# Calculate rolling statistics
rolling_window = 12  # 12 months

# Select a few key exposures for clarity
selected_exposures = ['us_large_equity', 'broad_ust', 'real_estate', 'gold']
selected_exposures = [exp for exp in selected_exposures if exp in real_returns_df.columns]

# Create subplots
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Rolling 12-Month Returns', 'Rolling Volatility', 'Rolling Sharpe Ratio'),
    vertical_spacing=0.08,
    shared_xaxes=True
)

for exp_id in selected_exposures:
    # Rolling returns
    rolling_returns = real_returns_df[exp_id].rolling(rolling_window).mean() * 12 * 100
    fig.add_trace(
        go.Scatter(x=rolling_returns.index, y=rolling_returns, name=exp_id, mode='lines'),
        row=1, col=1
    )
    
    # Rolling volatility
    rolling_vol = real_returns_df[exp_id].rolling(rolling_window).std() * np.sqrt(12) * 100
    fig.add_trace(
        go.Scatter(x=rolling_vol.index, y=rolling_vol, name=exp_id, mode='lines', showlegend=False),
        row=2, col=1
    )
    
    # Rolling Sharpe ratio
    rolling_sharpe = (rolling_returns / 100) / (rolling_vol / 100)
    fig.add_trace(
        go.Scatter(x=rolling_sharpe.index, y=rolling_sharpe, name=exp_id, mode='lines', showlegend=False),
        row=3, col=1
    )

# Add zero line to returns and Sharpe
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=3)

fig.update_layout(
    height=900,
    title_text="Rolling Performance Metrics (12-Month Window)",
    hovermode='x unified'
)

fig.update_yaxes(title_text="Annual Return (%)", row=1)
fig.update_yaxes(title_text="Annual Volatility (%)", row=2)
fig.update_yaxes(title_text="Sharpe Ratio", row=3)
fig.update_xaxes(title_text="Date", row=3)

fig.show()

## 12. Drawdown Analysis

In [15]:
# Calculate drawdownsdef calculate_drawdown(returns):    """Calculate drawdown series from returns."""    cumulative = (1 + returns).cumprod()    running_max = cumulative.expanding().max()    drawdown = (cumulative - running_max) / running_max    return drawdown# Calculate drawdowns for selected exposuresdrawdowns = {}for exp_id in selected_exposures:    if exp_id in real_returns_df.columns:        drawdowns[exp_id] = calculate_drawdown(real_returns_df[exp_id])# Create the plotfig = go.Figure()for exp_id, dd in drawdowns.items():    fig.add_trace(        go.Scatter(            x=dd.index,            y=dd * 100,            name=exp_id,            mode='lines',            fill='tozeroy'        )    )fig.update_layout(    title="Drawdown Analysis (Real Returns)",    xaxis_title="Date",    yaxis_title="Drawdown (%)",    height=500,    hovermode='x unified')fig.update_yaxes(autorange="reversed")  # Drawdowns are negativefig.show()# Calculate maximum drawdownsprint("\n📉 Maximum Drawdowns:")max_drawdowns = pd.DataFrame({    exp_id: {        'Max Drawdown': dd.min() * 100,        'Max DD Date': dd.idxmin(),        'Recovery Days': len(dd[dd < 0]) if len(dd[dd < 0]) > 0 else 0    }    for exp_id, dd in drawdowns.items()}).Tprint(max_drawdowns.round(1))

## 13. Efficient Frontier Preview

In [16]:
# Use the return estimation framework to get proper estimates
estimation_framework = ReturnEstimationFramework(total_return_fetcher, fred_fetcher)

# Get real return estimates and covariance matrix
try:
    # Get a subset of exposures for cleaner visualization
    demo_exposures = ['us_large_equity', 'us_small_equity', 'broad_ust', 'real_estate', 'gold']
    demo_universe = ExposureUniverse()
    
    for exp_id in demo_exposures:
        exposure = universe.get_exposure(exp_id)
        if exposure:
            demo_universe.exposures[exp_id] = exposure
    
    # Estimate returns
    returns_est, impl_info = estimation_framework.estimate_real_returns(
        demo_universe, start_date, end_date, method="historical", frequency="monthly"
    )
    
    # Get covariance matrix
    if not real_returns_df[demo_exposures].dropna().empty:
        cov_matrix = estimation_framework.estimate_covariance_matrix(
            real_returns_df[demo_exposures].dropna(), 
            method="sample", 
            frequency="monthly"
        )
        
        # Calculate individual asset risk/return
        individual_returns = returns_est
        individual_vols = np.sqrt(np.diag(cov_matrix))
        
        # Generate efficient frontier points (simplified)
        n_portfolios = 1000
        n_assets = len(demo_exposures)
        
        # Random portfolio weights
        weights = np.random.dirichlet(np.ones(n_assets), n_portfolios)
        
        # Calculate portfolio returns and risks
        portfolio_returns = weights @ individual_returns
        portfolio_risks = np.sqrt(np.diag(weights @ cov_matrix @ weights.T))
        
        # Create the plot
        fig = go.Figure()
        
        # Add random portfolios
        fig.add_trace(go.Scatter(
            x=portfolio_risks * 100,
            y=portfolio_returns * 100,
            mode='markers',
            marker=dict(
                size=3,
                color=portfolio_returns / portfolio_risks,  # Sharpe ratio
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Sharpe<br>Ratio")
            ),
            name='Random Portfolios',
            text=[f'Sharpe: {(r/v):.2f}' for r, v in zip(portfolio_returns, portfolio_risks)],
            hovertemplate='Risk: %{x:.1f}%<br>Return: %{y:.1f}%<br>%{text}'
        ))
        
        # Add individual assets
        fig.add_trace(go.Scatter(
            x=individual_vols * 100,
            y=individual_returns * 100,
            mode='markers+text',
            marker=dict(size=15, color='red', symbol='star'),
            text=demo_exposures,
            textposition='top center',
            name='Individual Assets'
        ))
        
        fig.update_layout(
            title="Efficient Frontier Preview (Real Returns)",
            xaxis_title="Annual Volatility (%)",
            yaxis_title="Annual Return (%)",
            height=600,
            hovermode='closest'
        )
        
        fig.show()
        
        print("\n📊 Individual Asset Statistics (Real Returns):")
        asset_stats = pd.DataFrame({
            'Annual Return': individual_returns * 100,
            'Annual Volatility': individual_vols * 100,
            'Sharpe Ratio': individual_returns / individual_vols
        })
        print(asset_stats.round(2))
        
except Exception as e:
    print(f"Error creating efficient frontier: {e}")
    import traceback
    traceback.print_exc()


📊 Individual Asset Statistics (Real Returns):
                 Annual Return  Annual Volatility  Sharpe Ratio
us_large_equity         9.3700            16.3600        0.5700
us_small_equity         6.2100            21.4200        0.2900
broad_ust              -6.1300             6.2800       -0.9800
real_estate             3.6700            19.4800        0.1900
gold                    7.1800            14.6300        0.4900


## 14. Export Results

In [17]:
# Create results directory
import os
results_dir = '../results/exposure_analysis'
os.makedirs(results_dir, exist_ok=True)

# Export summary statistics
summary_export = pd.DataFrame({
    'Exposure': real_stats.index,
    'Category': real_stats['Category'],
    'Nominal_Annual_Return': nominal_stats['Ann. Return'] * 100,
    'Real_Annual_Return': real_stats['Ann. Return'] * 100,
    'Inflation_Impact': (nominal_stats['Ann. Return'] - real_stats['Ann. Return']) * 100,
    'Annual_Volatility': real_stats['Ann. Volatility'] * 100,
    'Sharpe_Ratio': real_stats['Sharpe Ratio'],
    'Skewness': real_stats['Skewness'],
    'Kurtosis': real_stats['Kurtosis'],
    'Implementation': [universe_returns[exp]['implementation'] for exp in real_stats.index]
})

# Save to CSV
summary_export.to_csv(f'{results_dir}/exposure_summary_stats.csv', index=False)
print(f"✅ Exported summary statistics to {results_dir}/exposure_summary_stats.csv")

# Export returns data
real_returns_df.to_csv(f'{results_dir}/real_returns_{frequency}.csv')
print(f"✅ Exported real returns to {results_dir}/real_returns_{frequency}.csv")

# Export correlation matrix
correlation_matrix.to_csv(f'{results_dir}/correlation_matrix.csv')
print(f"✅ Exported correlation matrix to {results_dir}/correlation_matrix.csv")

# Save key metrics
with open(f'{results_dir}/analysis_summary.txt', 'w') as f:
    f.write("EXPOSURE UNIVERSE ANALYSIS SUMMARY\n")
    f.write("=" * 50 + "\n\n")
    f.write(f"Analysis Period: {start_date.date()} to {end_date.date()}\n")
    f.write(f"Data Frequency: {frequency}\n")
    f.write(f"Inflation Series: {inflation_series}\n")
    f.write(f"\nTotal Exposures: {len(universe)}\n")
    f.write(f"Successful Data Fetches: {successful}/{len(universe)}\n")
    f.write(f"\nAverage Real Return: {real_stats['Ann. Return'].mean() * 100:.2f}%\n")
    f.write(f"Average Volatility: {real_stats['Ann. Volatility'].mean() * 100:.2f}%\n")
    f.write(f"Average Sharpe Ratio: {real_stats['Sharpe Ratio'].mean():.2f}\n")
    f.write(f"\nAverage Inflation Impact: {(nominal_stats['Ann. Return'] - real_stats['Ann. Return']).mean() * 100:.2f}%\n")
    f.write(f"Period Inflation Rate: {inflation_rates.mean():.2%} (annualized)\n")

print(f"✅ Exported analysis summary to {results_dir}/analysis_summary.txt")
print("\n📁 All results exported successfully!")

✅ Exported summary statistics to ../results/exposure_analysis/exposure_summary_stats.csv
✅ Exported real returns to ../results/exposure_analysis/real_returns_monthly.csv
✅ Exported correlation matrix to ../results/exposure_analysis/correlation_matrix.csv
✅ Exported analysis summary to ../results/exposure_analysis/analysis_summary.txt

📁 All results exported successfully!


## Summary and Next Steps

This notebook demonstrated:

1. **Data Retrieval**: Successfully fetched total returns for all exposures in the universe
2. **Inflation Integration**: Retrieved CPI data from FRED and calculated inflation rates
3. **Real Return Conversion**: Converted nominal returns to real returns using exact method
4. **Comprehensive Analysis**: 
   - Summary statistics for all exposures
   - Performance comparisons (nominal vs real)
   - Correlation analysis
   - Rolling performance metrics
   - Drawdown analysis
   - Efficient frontier preview

### Key Findings:
- The infrastructure successfully handles all exposure types including rate series
- Real returns show the significant impact of inflation on nominal performance
- Different asset classes show varying sensitivity to inflation
- The data is ready for portfolio optimization

### Next Steps:
1. **Portfolio Optimization**: Use the real returns and covariance estimates in the optimization engine
2. **Backtesting**: Test various portfolio strategies using historical data
3. **Risk Analysis**: Deeper dive into factor exposures and risk contributions
4. **Strategy Development**: Develop and test systematic allocation strategies
5. **Web Interface**: Build interactive dashboards for real-time monitoring